In [2]:
import numpy as np
import keras
import tensorflow
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Dropout, MaxPooling2D, Reshape
from keras.models import Sequential, Model
from keras.utils import to_categorical, normalize
from IPython.display import HTML
import pygame , sys

pygame 2.2.0 (SDL 2.0.22, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
#Constants
WIDTH, HEIGHT = 400, 400
TITLE = "Smooth Movement"

#Player Class
class Player:
    def __init__(self):
        self.x = np.random.randint(-50,50) + 50
        self.y = np.random.randint(-50,50) + 50
        self.speed = 1
        self.WASD = [False,False,False,False]
    
    def update(self):
        if self.WASD[0] and not self.WASD[2]:
            self.y += self.speed #np.random.randint(1,5)
        if self.WASD[1] and not self.WASD[3]:
            self.x -= self.speed #np.random.randint(1,5)
        if self.WASD[2] and not self.WASD[0]: 
            self.y -= self.speed #np.random.randint(1,5)
        if self.WASD[3] and not self.WASD[1]: 
            self.x += self.speed #np.random.randint(1,5)

    def draw(self):
        self.rect = pygame.Rect(self.x, self.y, 32, 32)
        pygame.draw.rect(win, (250, 120, 60) , self.rect)

class Objective:
    def __init__(self):
        self.x = np.random.randint(-50,50) + 50
        self.y = np.random.randint(-50,50) + 50

    def draw(self):
        self.rect = pygame.Rect(self.x, self.y, 32, 32)
        pygame.draw.rect(win, (250, 120, 60) , self.rect)

player = Player()
objective = Objective()
print("Player: " + str((player.x,player.y)) + " Objective: " + str((objective.x,objective.y)))


Player: (85, 20) Objective: (73, 48)


In [9]:

X = []
y = []
step = 0
for _ in range(10000):
# while True:
    Action = np.random.randint(0,4)
    action_onehot = to_categorical(Action, num_classes=4)
    X.append(player.WASD + [objective.x - player.x, objective.y - player.y])
    y.append(action_onehot)
    
    if Action == 0:
        player.WASD[0] = True
        player.WASD[not 0] = False
    if Action == 1:
        player.WASD[1] = True
        player.WASD[not 1] = False
    if Action == 2:
        player.WASD[2] = True
        player.WASD[not 2] = False
    if Action == 3:
        player.WASD[3] = True
        player.WASD[not 3] = False
    
    player.update()
    step += 1

    if (player.x,player.y) ==  (objective.x,objective.y):
        break

print("Step: "+ str(step) +"Player: " + str((player.x,player.y)) + " Objective: " + str((objective.x,objective.y)))
X = np.array(X)
y = np.array(y)


Step: 10000Player: (969985, -1452551) Objective: (87, 79)


In [11]:

#Constants
WIDTH, HEIGHT = 400, 400
TITLE = "Smooth Movement"
global win
#pygame initialization
pygame.init()
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption(TITLE)
clock = pygame.time.Clock()

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        Action = np.random.randint(0,4)
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_w:
                Action = 0  
            if event.key == pygame.K_a:
                Action = 1  
            if event.key == pygame.K_s:
                Action = 2 
            if event.key == pygame.K_d:
                Action = 3   
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_w:
                Action = 4
            if event.key == pygame.K_a:
                Action = 4
            if event.key == pygame.K_s:
                Action = 4
            if event.key == pygame.K_d:
                Action = 4  

        if Action == 0:
            player.WASD[0] = True
            player.WASD[not 0] = False
        if Action == 1:
            player.WASD[1] = True
            player.WASD[not 1] = False
        if Action == 2:
            player.WASD[2] = True
            player.WASD[not 2] = False
        if Action == 3:
            player.WASD[3] = True
            player.WASD[not 3] = False
        
    #Draw
    win.fill((12, 24, 36))  
    player.draw()
    objective.draw()

    #update
    player.update()
    pygame.display.flip()

    clock.tick(120)

SystemExit: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
input_shape = X[0].shape

# Create your Model
model = Sequential()
model.add(Reshape(input_shape, input_shape=input_shape))
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(4, activation='softmax'))

# Compile model
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          epochs=50,
          batch_size=10, 
          validation_data=(X_test, y_test))

Epoch 1/50
800/800 [==============================] - 4s 4ms/step - loss: 2.9190 - accuracy: 0.2541 - val_loss: 0.5640 - val_accuracy: 0.2445
Epoch 2/50
800/800 [==============================] - 2s 3ms/step - loss: 0.5735 - accuracy: 0.2491 - val_loss: 0.5624 - val_accuracy: 0.2445
Epoch 3/50
800/800 [==============================] - 2s 3ms/step - loss: 0.5667 - accuracy: 0.2474 - val_loss: 0.5624 - val_accuracy: 0.2580
Epoch 4/50
800/800 [==============================] - 2s 3ms/step - loss: 0.5638 - accuracy: 0.2564 - val_loss: 0.5623 - val_accuracy: 0.2580
Epoch 5/50
800/800 [==============================] - 2s 2ms/step - loss: 0.5652 - accuracy: 0.2491 - val_loss: 0.5624 - val_accuracy: 0.2500
Epoch 6/50
800/800 [==============================] - 2s 2ms/step - loss: 0.5641 - accuracy: 0.2503 - val_loss: 0.5624 - val_accuracy: 0.2500
Epoch 7/50
800/800 [==============================] - 2s 2ms/step - loss: 0.5637 - accuracy: 0.2512 - val_loss: 0.5624 - val_accuracy: 0.2500
Epoch 

KeyboardInterrupt: 

In [ ]:
player = Player()
objective = Objective()
while True:
    state = player.WASD + [objective.x - player.x, objective.y - player.y]
    action_probs = model.predict(np.array([state]))[0]
    action = np.argmax(action_probs)
    
    if action == 0:
        player.WASD[0] = True
        player.WASD[not 0] = False
    elif action == 1:
        player.WASD[1] = True
        player.WASD[not 1] = False
    elif action == 2:
        player.WASD[2] = True
        player.WASD[not 2] = False
    elif action == 3:
        player.WASD[3] = True
        player.WASD[not 3] = False
    
    player.update()
    print("Player: " + str((player.x,player.y)) + " Objective: " + str((objective.x,objective.y)))
    
    if player.x == objective.x and player.y == objective.y:
        print("Objective reached!")
        break

1/1 [==============================] - 0s 76ms/step
Player: (-11, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 33ms/step
Player: (-10, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 39ms/step
Player: (-9, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 38ms/step
Player: (-8, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 29ms/step
Player: (-7, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 28ms/step
Player: (-6, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 24ms/step
Player: (-5, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 44ms/step
Player: (-4, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 32ms/step
Player: (-3, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 40ms/step
Player: (-2, 26) Objective: (-7, 34)
1/1 [==============================] - 0s 49ms/step
Player: (-1, 26) Objective: (-7, 34)
1/1 [==============

KeyboardInterrupt: 